

# Saving/Resuming Study with RDB Backend

An RDB backend enables persistent experiments (i.e., to save and resume a study) as well as access to history of studies.
In addition, we can run multi-node optimization tasks with this feature, which is described in `distributed`.

In this section, let's try simple examples running on a local environment with SQLite DB.

<div class="alert alert-info"><h4>Note</h4><p>You can also utilize other RDB backends, e.g., PostgreSQL or MySQL, by setting the storage argument to the DB's URL.
    Please refer to `SQLAlchemy's document <https://docs.sqlalchemy.org/en/latest/core/engines.html#database-urls>`_ for how to set up the URL.</p></div>


## New Study

We can create a persistent study by calling :func:`~optuna.study.create_study` function as follows.
An SQLite file ``example.db`` is automatically initialized with a new study record.


In [1]:
import logging
import sys

import optuna

# Add stream handler of stdout to show the messages
optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
study_name = "example-study"  # Unique identifier of the study.
storage_name = "sqlite:///{}.db".format(study_name)
study = optuna.create_study(study_name=study_name, storage=storage_name)

[I 2021-08-27 10:39:14,388] A new study created in RDB with name: example-study


A new study created in RDB with name: example-study


To run a study, call :func:`~optuna.study.Study.optimize` method passing an objective function.



In [2]:
def objective(trial):
    x = trial.suggest_float("x", -10, 10)
    return (x - 2) ** 2


study.optimize(objective, n_trials=3)

[I 2021-08-27 10:40:13,465] Trial 0 finished with value: 15.801337181374793 and parameters: {'x': -1.9750895815534513}. Best is trial 0 with value: 15.801337181374793.


Trial 0 finished with value: 15.801337181374793 and parameters: {'x': -1.9750895815534513}. Best is trial 0 with value: 15.801337181374793.


[I 2021-08-27 10:40:13,778] Trial 1 finished with value: 20.56818788704258 and parameters: {'x': -2.535216410166397}. Best is trial 0 with value: 15.801337181374793.


Trial 1 finished with value: 20.56818788704258 and parameters: {'x': -2.535216410166397}. Best is trial 0 with value: 15.801337181374793.


[I 2021-08-27 10:40:14,067] Trial 2 finished with value: 5.042694667903596 and parameters: {'x': 4.245594502109318}. Best is trial 2 with value: 5.042694667903596.


Trial 2 finished with value: 5.042694667903596 and parameters: {'x': 4.245594502109318}. Best is trial 2 with value: 5.042694667903596.


## Resume Study

To resume a study, instantiate a :class:`~optuna.study.Study` object
passing the study name ``example-study`` and the DB URL ``sqlite:///example-study.db``.



In [3]:
study = optuna.create_study(study_name=study_name, storage=storage_name, load_if_exists=True)
study.optimize(objective, n_trials=3)

[I 2021-08-27 10:40:28,255] Using an existing study with name 'example-study' instead of creating a new one.


Using an existing study with name 'example-study' instead of creating a new one.


[I 2021-08-27 10:40:28,680] Trial 3 finished with value: 115.63720897559784 and parameters: {'x': -8.75347427465179}. Best is trial 2 with value: 5.042694667903596.


Trial 3 finished with value: 115.63720897559784 and parameters: {'x': -8.75347427465179}. Best is trial 2 with value: 5.042694667903596.


[I 2021-08-27 10:40:29,243] Trial 4 finished with value: 65.5740809526728 and parameters: {'x': -6.097782471311069}. Best is trial 2 with value: 5.042694667903596.


Trial 4 finished with value: 65.5740809526728 and parameters: {'x': -6.097782471311069}. Best is trial 2 with value: 5.042694667903596.


[I 2021-08-27 10:40:29,839] Trial 5 finished with value: 11.184848315594344 and parameters: {'x': -1.3443756241777542}. Best is trial 2 with value: 5.042694667903596.


Trial 5 finished with value: 11.184848315594344 and parameters: {'x': -1.3443756241777542}. Best is trial 2 with value: 5.042694667903596.


## Experimental History

We can access histories of studies and trials via the :class:`~optuna.study.Study` class.
For example, we can get all trials of ``example-study`` as:



In [4]:
study = optuna.create_study(study_name=study_name, storage=storage_name, load_if_exists=True)
df = study.trials_dataframe(attrs=("number", "value", "params", "state"))

[I 2021-08-27 10:40:44,117] Using an existing study with name 'example-study' instead of creating a new one.


Using an existing study with name 'example-study' instead of creating a new one.


The method :func:`~optuna.study.Study.trials_dataframe` returns a pandas dataframe like:



In [5]:
print(df)

   number       value  params_x     state
0       0   15.801337 -1.975090  COMPLETE
1       1   20.568188 -2.535216  COMPLETE
2       2    5.042695  4.245595  COMPLETE
3       3  115.637209 -8.753474  COMPLETE
4       4   65.574081 -6.097782  COMPLETE
5       5   11.184848 -1.344376  COMPLETE


A :class:`~optuna.study.Study` object also provides properties
such as :attr:`~optuna.study.Study.trials`, :attr:`~optuna.study.Study.best_value`,
:attr:`~optuna.study.Study.best_params` (see also `first`).



In [6]:
print("Best params: ", study.best_params)
print("Best value: ", study.best_value)
print("Best Trial: ", study.best_trial)
print("Trials: ", study.trials)

Best params:  {'x': 4.245594502109318}
Best value:  5.042694667903596
Best Trial:  FrozenTrial(number=2, values=[5.042694667903596], datetime_start=datetime.datetime(2021, 8, 27, 10, 40, 13, 815434), datetime_complete=datetime.datetime(2021, 8, 27, 10, 40, 13, 890104), params={'x': 4.245594502109318}, distributions={'x': UniformDistribution(high=10.0, low=-10.0)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=3, state=TrialState.COMPLETE, value=None)
Trials:  [FrozenTrial(number=0, values=[15.801337181374793], datetime_start=datetime.datetime(2021, 8, 27, 10, 40, 13, 171746), datetime_complete=datetime.datetime(2021, 8, 27, 10, 40, 13, 267375), params={'x': -1.9750895815534513}, distributions={'x': UniformDistribution(high=10.0, low=-10.0)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=1, state=TrialState.COMPLETE, value=None), FrozenTrial(number=1, values=[20.56818788704258], datetime_start=datetime.datetime(2021, 8, 27, 10, 40, 13, 505451), date

In [7]:
# MWB - added extra run
study = optuna.create_study(study_name=study_name, storage=storage_name, load_if_exists=True)
study.optimize(objective, n_trials=3)

[I 2021-08-27 10:44:26,312] Using an existing study with name 'example-study' instead of creating a new one.


Using an existing study with name 'example-study' instead of creating a new one.


[I 2021-08-27 10:44:26,729] Trial 6 finished with value: 38.73194516462048 and parameters: {'x': 8.223499430756018}. Best is trial 2 with value: 5.042694667903596.


Trial 6 finished with value: 38.73194516462048 and parameters: {'x': 8.223499430756018}. Best is trial 2 with value: 5.042694667903596.


[I 2021-08-27 10:44:27,070] Trial 7 finished with value: 14.599647057114916 and parameters: {'x': -1.8209484499420974}. Best is trial 2 with value: 5.042694667903596.


Trial 7 finished with value: 14.599647057114916 and parameters: {'x': -1.8209484499420974}. Best is trial 2 with value: 5.042694667903596.


[I 2021-08-27 10:44:27,379] Trial 8 finished with value: 20.71653396856652 and parameters: {'x': -2.5515419330779014}. Best is trial 2 with value: 5.042694667903596.


Trial 8 finished with value: 20.71653396856652 and parameters: {'x': -2.5515419330779014}. Best is trial 2 with value: 5.042694667903596.
